In [ ]:
import tensorflow as tf
import re
import gensim
import numpy as np
import pydot
import os
import json

from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation, Dropout, CuDNNLSTM, Input, Concatenate, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import plot_model, to_categorical

from gensim.models.wrappers import FastText

#from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras import backend as K
import tensorflow as tf


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
pres = ['hoover', 'fdroosevelt', 'truman', 'hayes', 'adams', 'carter', 'madison', 'ford', 'cleveland', 'obama', 'harding', 'wilson', 'taylor', 'monroe', 'lincoln', '.ipynb_checkpoints', 'jefferson', 'vanburen', 'jackson', 'washington', 'polk', 'bush', 'gwbush', 'pierce', 'reagan', 'garfield', 'mckinley', 'coolidge', 'roosevelt', 'fillmore', 'johnson', 'harrison', 'taft', 'lbjohnson', 'jqadams', 'tyler', 'clinton', 'kennedy', 'eisenhower', 'nixon', 'arthur', 'grant', 'buchanan', 'bharrison']

file_train = ''
file_val = ''
file_test = ''

for root, dirs, files in os.walk("../1.DataPreparationResults", topdown=False):

    for name in files:
        file = os.path.join(root, name)
        
        if root.split('/')[2] in pres:
            text = open(file).read()
            
            if 'train' in file:
                file_train += text
            elif 'val' in file:
                file_val += text
            elif 'test' in file:
                file_test += text 

# Write to the 2.GPT2 XLNET folder
dir_split = "../1.DataPreparationResults/combined_data"
open(f"{dir_split}/train.txt", 'w').write(file_train)
open(f"{dir_split}/val.txt", 'w').write(file_val)
open(f"{dir_split}/test.txt", 'w').write(file_test)

In [ ]:
# This essentially is to produce a unique words list from all the text before splitting into sentences

def google_preprocess(file):
    file2 = re.sub('\d', '#', file)
    file2 = re.sub(' a ', ' A ', file2)
    file2 = re.sub(' and ', ' And ', file2)
    file2 = re.sub(' of ', ' Of ', file2)
    file2 = re.sub(' to ', ' To ', file2)
    # Add spaces around <speech_sep>
    # Create a set of all words in file.txt but remove <speech_sep>
    unique_words = set(file2.replace("<speech_sep>", " <speech_sep> ").split())
    unique_words.remove("<speech_sep>")
    return file2, unique_words

file_train_google, unique_words_train = google_preprocess(file_train)
file_val_google, unique_words_val = google_preprocess(file_val)
file_test_google, unique_words_test = google_preprocess(file_test)

unique_words_all = unique_words_train.union(unique_words_val.union(unique_words_test))
print("total number of unique words: ",len(unique_words_all))

In [ ]:
x_len = 30
x_step = 1

def file_to_rolling_sentences(file):
    sentences = []
    sentences2 = []
    next_words = []
    list_words = []
    
    for speech in file.split("<speech_sep>"):
        list_words = speech.split()
        # I noticed the last speech has zero word 
        # because <speech_sep> is the last character
        if len(list_words) == 0:
            break
        
        # each row should have x_len + 1 (both input and target)
        for i in range(0,len(list_words)-x_len-1, x_step):
            sentences2 = [word for word in list_words[i: i + x_len + 1]]
            sentences.append(sentences2)
            
    return sentences

# train_sentences = file_to_sentences(file_train)
train_sentences = file_to_rolling_sentences(file_train_google)
val_sentences = file_to_rolling_sentences(file_val_google)

In [ ]:
def file_to_each_sentence(file):
    
    sentence_all = []
    
    for speech in file.split("<speech_sep>"):
        sentences = speech.split('.')
        
        for sentence in sentences:
            sentence_all.append(sentence.strip() + '.')
    
    return sentence_all

train_sentences = file_to_each_sentence(file_train_google)
val_sentences = file_to_each_sentence(file_val_google)

In [ ]:
number_words = 25
maxlength = 64

tokenizer = Tokenizer(nb_words=number_words)
tokenizer.fit_on_texts(train_sentences)
tokenized_sequences = tokenizer.texts_to_sequences(train_sentences)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
index_word = new_dict = dict([(value, key) for key, value in word_index.items()]) 

In [ ]:
train_data = pad_sequences(tokenized_sequences, maxlen=maxlength)

In [ ]:
train_x = train_data[:, :-1]
train_y = train_data[:, -1]

In [ ]:
tokenized_val = tokenizer.texts_to_sequences(val_sentences)
val_data = pad_sequences(tokenized_val, maxlen=maxlength)
val_x = val_data[:, :-1]
val_y = val_data[:, -1]

### Prepare Word Embeddings

In [16]:
google_word_model = gensim.models.KeyedVectors.load_word2vec_format('../../test/GoogleNews-vectors-negative300.bin', binary=True)


In [ ]:
#get all the vectors
#pretrained_weights = google_word_model.wv.vectors

#word2idx
def word2idx(word):
    return google_word_model.wv.vocab[word].index
    

#idx2word
def idx2word(idx):
    return google_word_model.wv.index2word[idx]

#get vector of word
#google_word_model["Hi"]

In [ ]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = google_word_model[word] if word in google_word_model else None
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
# model = tf.keras.Sequential()

# inputs = Input(shape=(49,))

# embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=maxlength-1, trainable=False)(inputs)
# lstm1 = LSTM(units = 300, return_sequences = True)(embedding_layer)
# lstm2 = LSTM(units = 300, return_sequences = True)(lstm1)
# predictions = Dense(len(word_index), activation='softmax')(lstm2)


# #test = Concatenate(axis=1)([embedding_layer, lstm1, lstm2])
# #custom_layer = AttentionWeightedAverage(name='attention')(test)

# model = Model(inputs=inputs,outputs=predictions)

# # model.add(embedding_layer)
# # model.add(LSTM(units = 128, return_sequences = True))
# # model.add(LSTM(units = 128, return_sequences = True))
# # model.add(Dense(len(word_index), activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# #print(model.summary())
# #plot_model(model)

In [ ]:
embedding_layer = Embedding(len(word_index)+1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=maxlength-1, trainable=False)

model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(LSTM(units = 300, return_sequences=True))
model.add(LSTM(units = 300))
#model.add(Flatten())
model.add(Dense(len(word_index), activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
history = model.fit(train_x, train_y, epochs=10, batch_size = 64, validation_data=(val_x, val_y)) #, callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

In [23]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)


def generate_next(text, num_generated=50):
    tokenized_sequences = [word_index[word] for word in text.lower().split() if word in word_index.keys()]
    #tokenized_sequences = [word2idx(word) for word in text.lower().split() if word in google_word_model]
    
    #maxlen = maxlength, was 33 before.
    padded_sequences = pad_sequences([tokenized_sequences], maxlen=maxlength+1)[:, 1:]
    
    #print(padded_sequences)
    
    generated_tokens = []
    
    for i in range(num_generated):
        #print(padded_sequences.shape)
        prediction = model.predict(x=padded_sequences)
        idx = sample(prediction[-1], temperature=0.9)
        
        generated_tokens.append(idx)
        
        tokenized_sequences = np.append(tokenized_sequences, idx)
        padded_sequences = pad_sequences([tokenized_sequences], maxlen=maxlength+1)[:, 1:]
        
        
    return ' '.join([index_word[idx] if idx != 0 else "</unk>" for idx in generated_tokens])
    
    #return ' '.join([idx2word(idx) for idx in padded_sequences[0] if idx != 0])

def on_epoch_end(epoch, _):
    #print('\nGenerating text after epoch: %d' % epoch)
    
    texts = ["There are two ways to love you"]
    
    for text in texts:
        sample = generate_next(text)
        print('%s... -> %s' % (text, sample))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
generate_next("Hello How are you")

In [ ]:
with open('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.json', 'w') as f:
    json.dump(str(history.history), f)

In [ ]:
model.save('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.h5') 

In [14]:
model = tf.keras.models.load_model('train_history_sentence_style_google_w2v_embedtrain_F_LSTM2x.h5')

# Check its architecture
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 64, 300)           7125900   
_________________________________________________________________
lstm_25 (LSTM)               (None, 64, 300)           721200    


In [24]:
generate_next("Hello How are you")

'as of of of to of our the to not the in not that on that the our be and to a of the have this to of of that the the of for the of the in of with the of the for to the the to have a'

In [25]:
fh = open("LSTM-test-file.txt").read().splitlines()

In [26]:
output = open("LSTM-fixed-window-result.txt", 'w')

In [ ]:
for i,line in enumerate(fh):
    print("###############################")
    prediction = generate_next(line)
    actual = fh[i+1]
    
    output.write(line + '|' + actual + '|' + prediction)
    print(prediction)
    print(actual)

###############################
the the the to our to of to and a this is the to the of this this of the the of which for our the of the with our our the a to the the to a for of which to it of this the to the the of
Through its mysterious channels we come to wider acquaintance with surroundings and men.
###############################
to to this of are our this of the we our of be and with as of of be the a of i a the of to of our in the of will to the to the our and of that the this not is in the a that it
The microphone for these few moments has been brought to the President s study in the East Wing of the White House.
###############################
of to the of the a to a that a the not to i of of to of will the as by our of the are the to for of of our are to the of a of for this a the it is the of the the for we
This room from which I speak was the scene of work and accomplishment of our Presidents for over a century.
###############################
we with the the of the of for

the of to the is of by is that to of of a the that of to a a our that have which the with </unk> </unk> a to of the to with we of a of the the is to a a it have of have with of the
That is the reduction of arms.
###############################
in it is a the to of that to the is the that will and the our this </unk> the the to by of of to of of a </unk> the of that by to of the of is have will of be to of a for the the a
Some months ago I proposed to the world that we should further reduce and limit naval arms.
###############################
of for the the the the to not is are of and the to of the to for be the that of of of i and are of the the the of on to our the this the that that this of of of the the the in of to
Today we are engaged in a most hopeful discussion with other governments leading to this end.
###############################
for by a to will the in the for in our </unk> of the of our be to a is of the and we to the this the the of be the a the of that the on to a th

for to and the will be have a to are are our and of the of this this with </unk> of </unk> to of the are a be of the the the the of by the a the of be a in to the of the a in to of
In that case there was no Federal Reserve System.
###############################
a not is </unk> be of </unk> for this of to of in to to be a with of of the that of a of the </unk> a with is to be the the that of of it to and and in </unk> that as the the to is in
There was no way to acquaint of capital movement over the country , and the interest rates ran up to 300 percent.
###############################
to as have we that a of the in of which is a of be the that of of not to of the it of a a with the of of and have of to to our for the the the our the of this our have the are the
The result was to bring about a monetary panic in the entire country.
###############################
our the a the to the to of </unk> is the i of of of the to i by to will to of the the to be it the to have that of this of th

the the of the with in on not and with the the the of a the are our to of and will by in the be to is of </unk> to be the to of this to to the to be </unk> i of the we the and the of
And that is about a summary of the whole situation as it stands at this moment.
###############################
not are have of that of to of in and to of of to the by our a of the a and of to of the with this will to of of of a have have have the not our of a with of a and it our our have
.
###############################
is of a to of for be of this with are it </unk> we the the a be of the </unk> the the the of is of a will i for a of by in the to the our the of the a that to of not the to to
We are dealing here with a psychological situation to a very considerable degree.
###############################
of are by a are and </unk> a the the that the of of a to of the to a of the the in the for the the is is our to the a that to the a the and and the </unk> our a of a of to the
It is a question of fear.


that the a in of that of the for of </unk> of i of to will that have the for to of of the the for to to we by we will by the the the with the of of is the a the the the our is to of
It is a dedication and consecration under God to the highest office in service of our people.
###############################
to a the the the have a our the of be this in the to and the we of and the the of of of not the of a to we which the to to the the the in of not the of that to this to of to to
I assume this trust in the humility of knowledge that only through the guidance of Almighty Providence can I hope to discharge its ever increasing burdens.
###############################
the the have be to to of for our is the of to with </unk> of of the for with and and of the by the the this of be the in it we the of that by of on of with our and by of of the our the
It is in keeping with tradition throughout our history that I should express simply and directly the opinions which I hold concerning some of 

...

the the a the the this </unk> to to to of not be of of of to a of to a which the the the of of this and the the to of of and and to have of be is the a to is be we the the of
The rules , as they have hitherto been enforced , have resulted beneficially , as is shown by the opinions of the members of the Cabinet and their subordinates in the Departments , and in that opinion I concur.
###############################
for our a the of of are of of of are and the a a this of have the of of of this to of the of the in the of the our the we the we of of by the be of this of of the the for of
And in the annual message of December of the same year similar views are expressed and an appropriation for continuing the work of the Commission again advised.
###############################
and our of a of be our to that it to a </unk> a a the to of in to we to the are the the a the the have of and for our we that of our a the have of to for have this for of of in
The appropriation was not made , and a

on the the our of to have have with the for our of the with </unk> with the and we that for to and that the the of to a of to this and not of the our the for the to of the to the of to the is
It may not be easy , and it may never perhaps be necessary , to define with precision the proper limit of political action on the part of Federal officers.
###############################
the the the to and the in are the be a the a the this we our to the is to the the for be that of the that our the the the the of the the that will it will with the that a for the the with that
But while their right to hold and freely express their opinions can not be questioned , it is very plain that they should neither be allowed to devote to other subjects the time needed for the proper discharge of their official duties nor to use the authority of their office to enforce their own opinions or to coerce the political action of those who hold different opinions.
###############################
in the in the tha

the the be are of of will of of of have the be to a have to by have the the to the of the to of to a in the of the the a the and the of of by by of </unk> to is of be the the
An important reduction of the present rates of cable communication with Europe , felt to be too burdensome to the interests of our commerce , must necessarily flow from the establishment of this competing line.
###############################
the the i a a is is in a the to have by of and to be the the of </unk> it and of to the to to as of be to have it the for </unk> that it by the to to the of to of and the of
The attention of Congress was drawn to the propriety of some general regulation by Congress of the whole subject of transmarine cables by my predecessor in his message of December 7 , 1875 , and I respectfully submit to your consideration the importance of Congressional action in the matter.
###############################
in have in not a of the the to that will to we of be the and and a of by which is t

to our a of of the in to our of of of to of to of on the in of the that the of of of a a a for to of is the we and to the that the that of in of the of in the that by
The negotiations are still proceeding , and will be pressed with diligence.
###############################
be of in this a of that not </unk> the in it our in to as of is to the that for our of and of of of of and the of to a the it is which a the for our of our the our a and we be
A question having arisen between China and Japan about the Lew Chew Islands , the United States Government has taken measures to inform those powers of its readiness to extend its good offices for the maintenance of peace if they shall mutually deem it desirable and find it practicable to avail themselves of the proffer.
###############################
of to a of to and the we be of a the the and the to of of our to and to and that our of this we the the of with for the with the and it not of the the our to our the of of it a
It is a gratifica

of of of in not of have of of the to not our the of of the of a a by will a our that to a of </unk> the of to </unk> the to of that a to have of the that with of a by to a </unk>
The subject of opening diplomatic relations with Roumania and Servia , now become independent sovereignties , is at present under consideration , and is the subject of diplomatic correspondence.
###############################
to we of </unk> for of a of the to the to by are of be of a in will in with to a the of this on is a have by this we to by the of a to the we of it of in of be the a
There is a gratifying increase of trade with nearly all European and American countries , and it is believed that with judicious action in regard to its development it can and will be still more enhanced and that American products and manufactures will find new and expanding markets.
###############################
to in a the and the in our i of to the of of to of that </unk> is of the a of our the for to of to of be to the